# Adaptive Tensor Genetic Evolutionary Network

## Introduction

The **Adaptive Tensor Genetic Evolutionary Network (ATGEN)** is an innovative approach in neural network design that combines traditional neural networks with genetic algorithms. The core idea is to dynamically evolve the structure and parameters of the neural network during training, thereby creating a highly adaptable model capable of learning complex patterns more efficiently.

### Key Features

1. **Dynamic Network Evolution**: The network's structure is not fixed; it can evolve over time. Layers and neurons can be added or removed based on a set of evolutionary rules, which allows the network to adapt its complexity to the task at hand.

2. **Genetic Algorithm-Based Training**: The network employs a genetic algorithm to evolve its weights and structure. This involves selecting parent networks based on their fitness, performing crossover to combine features of two parents, and applying mutations to introduce new variations.

3. **Adaptive Activation Functions**: The network supports the dynamic adjustment of activation functions. This adaptability allows the model to switch from linear to non-linear transformations seamlessly, providing greater flexibility in learning different types of data distributions.

4. **Neuron and Weight Management**: ATGEN includes mechanisms to add and remove neurons and weights in each layer. This helps in optimizing the network size and performance during the evolution process.

## Implementation Overview

### Components

- **Linear Layer (`Linear` class)**: A custom linear layer with dynamic neuron management, including methods to add, remove, and initialize neurons and weights.
- **Activation Switch (`ActiSwitch` class)**: A module that allows smooth transitions between different activation functions, controlled by a learnable parameter.
- **Evolutionary Network (`ATNetwork` class)**: A neural network that evolves its structure and weights based on predefined rules and random mutations.
- **Genetic Algorithm (`GeneticAlgorithm` class)**: A framework that orchestrates the evolutionary process, handling fitness evaluation, selection, crossover, mutation, and pruning of neurons.

### Getting Started

Below, we provide a step-by-step implementation of the Adaptive Tensor Genetic Evolution Network and demonstrate its unique capabilities through various experiments and tests. The notebook is divided into sections covering the core components, training routines, and network evolution strategies.

---

Let's dive into the implementation details and see how we can build a network that learns to evolve!

In [211]:
# Import necessary libraries
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import AdamW

# Import utility packages
import math
import copy
import random
from typing import List, Tuple

# Import custom modules from the same folder
from atgen.layers import Linear, ActiSwitch, Pass
from atgen.network import ATNetwork
from atgen.memory import ReplayBuffer
from atgen.utils import activation_functions
from atgen.ga import ATGEN

# Additional visualization or utility packages, if needed
# from torchsummary import summary
# import gymnasium as gym


## The `Layer` Module

### 1. `ActiSwitch` with ReLU Activation

The `ActiSwitch` class is tested with the `nn.Tanh()` activation function:

- **Input**: A tensor of shape `(5, 4, 3, 2)` with random values.
- **Output**: The resulting tensor after applying the activation function.
- **Assertions**:
  - The shape of the output tensor should match the shape of the input tensor.
  - Check if the input and output tensors have the same values (i.e., the output tensor should be equal to the input tensor).

In [212]:
# 1. Testing ActiSwitch with ReLU Activation
linear_pass_relu = ActiSwitch(nn.Tanh())
x = torch.randn(5, 4, 3, 2)
output: torch.Tensor = linear_pass_relu(x)

print(f"Input: {x.shape}")
print(f"Output: {output.shape}")
print(f"No change in data: {(x==output).all().item()}")

Input: torch.Size([5, 4, 3, 2])
Output: torch.Size([5, 4, 3, 2])
No change in data: True


### 2. `Linear` Layer Initialization

The `Linear` layer is tested for its weight and bias initialization:

- **Initial State**:
  - Display the initial weights and biases of the `Linear` layer.

In [213]:
# 2. Testing Linear Layer Initialization
linear = Linear(in_features=3, out_features=2)

print("Initial weights:")
print(linear.weight)
print("Initial bias:")
print(linear.bias)

Initial weights:
Parameter containing:
tensor([[ 0.3527,  0.2342, -0.2040],
        [-0.2726, -0.5544,  0.1190]], requires_grad=True)
Initial bias:
Parameter containing:
tensor([ 0.3825, -0.4965], requires_grad=True)


### 3. Adding a Neuron to the `Linear` Layer

Test the addition of a new output neuron:

- **Action**: Add a new output neuron to the `Linear` layer.
- **Assertions**:
  - Display the updated weights and biases to verify the addition.

In [214]:
# 3. Adding a Neuron
linear.add_neuron()
print("After adding a new output neuron:")
print(linear.weight)
print(linear.bias)

After adding a new output neuron:
Parameter containing:
tensor([[ 0.3527,  0.2342, -0.2040],
        [-0.2726, -0.5544,  0.1190],
        [ 0.1701, -0.3570,  0.4331]], requires_grad=True)
Parameter containing:
tensor([ 0.3825, -0.4965, -0.5423], requires_grad=True)


### 4. Increasing the Input Dimension of the `Linear` Layer

Test the increase of the input dimension:

- **Action**: Increase the input dimension of the `Linear` layer.
- **Assertions**:
  - Display the updated weights and biases to verify the change in input dimension.

In [215]:
# 4. Increasing Input Dimension
linear.add_weight()
print("After increasing the input dimension:")
print(linear.weight)
print(linear.bias)

After increasing the input dimension:
Parameter containing:
tensor([[ 0.3527,  0.2342, -0.2040,  0.0000],
        [-0.2726, -0.5544,  0.1190,  0.0000],
        [ 0.1701, -0.3570,  0.4331,  0.0000]], requires_grad=True)
Parameter containing:
tensor([ 0.3825, -0.4965, -0.5423], requires_grad=True)


### 5. Removing a Neuron from the `Linear` Layer

Test the removal of an output neuron:

- **Action**: Remove an output neuron from the `Linear` layer.
- **Assertions**:
  - Display the updated weights and biases to verify the removal.

In [216]:
# 5. Removing a Neuron
linear.remove_neuron(0)  # Assumes such a method exists
print("After removing an output neuron:")
print(linear.weight)
print(linear.bias)

After removing an output neuron:
Parameter containing:
tensor([[-0.2726, -0.5544,  0.1190,  0.0000],
        [ 0.1701, -0.3570,  0.4331,  0.0000]], requires_grad=True)
Parameter containing:
tensor([-0.4965, -0.5423], requires_grad=True)


### 6. Removing an Input Weight from the `Linear` Layer

Test the removal of an input weight:

- **Action**: Remove an input weight from the `Linear` layer.
- **Assertions**:
  - Display the updated weights and biases to verify the removal.

In [217]:
# 6. Removing an Input Weight
linear.remove_weight(0)  # Assumes such a method exists
print("After removing an input weight:")
print(linear.weight)
print(linear.bias)

After removing an input weight:
Parameter containing:
tensor([[-0.5544,  0.1190,  0.0000],
        [-0.3570,  0.4331,  0.0000]], requires_grad=True)
Parameter containing:
tensor([-0.4965, -0.5423], requires_grad=True)


### 7. Identity Initialization of the `Linear` Layer

Test the identity initialization of the `Linear` layer:

- **Action**: Initialize a `Linear` layer with identity weights.
- **Assertions**:
  - Display the weights and biases to verify the identity initialization.

In [218]:
# 7. Testing Identity Initialization
identity_layer = Linear.init_identity_layer(size=3)
print("Weights after identity initialization:")
print(identity_layer.weight)
print("Bias after identity initialization:")
print(identity_layer.bias)

Weights after identity initialization:
Parameter containing:
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], requires_grad=True)
Bias after identity initialization:
Parameter containing:
tensor([0., 0., 0.], requires_grad=True)


## Understanding the Identity Matrix in Neural Networks

### Identity Matrix

In the context of neural networks, the identity matrix is often used as an initialization method for weights in newly added layers. The identity matrix is a square matrix with ones on the main diagonal and zeros elsewhere. It has the property that when used in matrix multiplication, it does not alter the original matrix:

$$
I_n = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 1 
\end{bmatrix}
$$

where $I_n$ is the identity matrix of size $n \times n$. This matrix is called the identity matrix because it acts as a multiplicative identity in matrix multiplication.

### Matrix Multiplication with the Identity Matrix

When you multiply a matrix $A$ by an identity matrix $I$, the result is the matrix $A$ itself:

$$
A \times I = A
$$

For example, consider a matrix $A$:

$$
A = \begin{bmatrix}
a & b \\
c & d
\end{bmatrix}
$$

Multiplying $A$ by the $2 \times 2$ identity matrix $I$:

$$
A \times I_2 = \begin{bmatrix}
a & b \\
c & d
\end{bmatrix}
\times
\begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
=
\begin{bmatrix}
a & b \\
c & d
\end{bmatrix}
$$

The result is $A$, demonstrating that the identity matrix does not change the matrix it multiplies.

### Usage in Neural Networks

In neural networks, initializing weights with an identity matrix ensures that newly added layers do not alter the input significantly when first added. This can be particularly useful when experimenting with network architectures or adding new neurons, as it maintains stability in the output until training adjusts the weights.

## Handling Activation Functions in Deep Networks

As neural networks become deeper, the introduction of new layers can affect the output of the network due to the activation functions applied. Even if the new layers are initialized with identity matrices, which theoretically do not alter the output directly, activation functions introduce non-linearity that can change the network's behavior.

### The Problem with Activation Functions

When adding new layers with identity matrix initialization, the direct output of these layers may remain unchanged. However, if the activation functions applied to these layers are non-linear, they can alter the output in ways that affect the network's overall performance. This introduces a challenge: ensuring that the output remains consistent while still benefiting from the added layers.

### Solution: Smooth Transition with Blending Activation Functions

To address this issue, a custom approach is used to blend linear and non-linear transformations. The idea is to smoothly transition between a linear pass-through and a non-linear activation function, based on a learnable weighting factor. This ensures that new layers can be added without introducing abrupt changes in output.

One effective solution is to use a class that blends linear and non-linear activations. This class provides a learnable parameter that adjusts the balance between the linear transformation and the non-linear activation function. By controlling this parameter, the network can maintain the output consistency while allowing the flexibility to benefit from the additional layers.

### Key Points

- **Identity Matrix**: Ensures that additional layers do not affect the output directly but does not account for changes introduced by activation functions.
- **Linear and Semi-Linear Activation Functions**: Maintain output consistency while adding layers, avoiding undesirable performance changes.
- **Blending Mechanism**: Allows a smooth transition between linear and non-linear functions, controlled by a learnable parameter, to ensure the network adapts effectively.

This approach helps in maintaining the performance and output consistency of deep networks while accommodating new layers and activation functions.
```​⬤

In [219]:
# Helper loss function
def loss(x1: torch.Tensor, x2: torch.Tensor):
    val = torch.abs((x2 / x1).mean() - 1) * 100
    print(f"loss = {val:.10f}%")

# Define the Custom Network
class CustomNetwork(ATNetwork):
    def __init__(self):
        # Do not call the parent __init__ with layers; initialize manually
        super(ATNetwork, self).__init__()
        self.layers = nn.ModuleList([
            Linear(5, 3),
            Linear(3, 1)
        ])
        self.activation = nn.ModuleList([
            ActiSwitch(),
            Pass()
        ])
        self.backprob_phase = True
        self.default_activation = nn.ReLU()

# Initialize the model and print the summary
model = CustomNetwork()
# summary(model, input_size=(5,))
model.summary()

# You can also initialize the model simply like next
model = ATNetwork([5, 3, 1])

# Test the network with random input
x = torch.randn(4, 5)
y1: torch.Tensor = model(x)

Model Summary:
-------------------------------------------------------------------------------------
Layer      Output Shape                  Parameters     Activation     
-------------------------------------------------------------------------------------
Layer 1    (batch_size, 3)               18             ActiSwitch(ReLU, 0.00%)
Layer 2    (batch_size, 1)               4              Pass           
-------------------------------------------------------------------------------------
Total Parameters:        22             


In [220]:
# Evolve the network by inserting new 10 layers
for _ in range(10):
    model.evolve_network()
# summary(model, input_size=(5,))
model.summary()

Model Summary:
-------------------------------------------------------------------------------------
Layer      Output Shape                  Parameters     Activation     
-------------------------------------------------------------------------------------
Layer 1    (batch_size, 5)               30             ActiSwitch(ReLU, 0.00%)
Layer 2    (batch_size, 5)               30             ActiSwitch(ReLU, 0.00%)
Layer 3    (batch_size, 5)               30             ActiSwitch(ReLU, 0.00%)
Layer 4    (batch_size, 5)               30             ActiSwitch(ReLU, 0.00%)
Layer 5    (batch_size, 5)               30             ActiSwitch(ReLU, 0.00%)
Layer 6    (batch_size, 5)               30             ActiSwitch(ReLU, 0.00%)
Layer 7    (batch_size, 5)               30             ActiSwitch(ReLU, 0.00%)
Layer 8    (batch_size, 3)               18             ActiSwitch(ReLU, 0.00%)
Layer 9    (batch_size, 3)               12             ActiSwitch(ReLU, 0.00%)
Layer 10   (batch_siz

In [221]:
# Evolve layers by adding new 500 nodes to random layers
for _ in range(500):
    model.evolve_layer()
# summary(model, input_size=(5,))
model.summary()

Model Summary:
-------------------------------------------------------------------------------------
Layer      Output Shape                  Parameters     Activation     
-------------------------------------------------------------------------------------
Layer 1    (batch_size, 44)              264            ActiSwitch(ReLU, 0.00%)
Layer 2    (batch_size, 60)              2700           ActiSwitch(ReLU, 0.00%)
Layer 3    (batch_size, 45)              2745           ActiSwitch(ReLU, 0.00%)
Layer 4    (batch_size, 49)              2254           ActiSwitch(ReLU, 0.00%)
Layer 5    (batch_size, 45)              2250           ActiSwitch(ReLU, 0.00%)
Layer 6    (batch_size, 42)              1932           ActiSwitch(ReLU, 0.00%)
Layer 7    (batch_size, 45)              1935           ActiSwitch(ReLU, 0.00%)
Layer 8    (batch_size, 45)              2070           ActiSwitch(ReLU, 0.00%)
Layer 9    (batch_size, 60)              2760           ActiSwitch(ReLU, 0.00%)
Layer 10   (batch_siz

In [222]:
# Print the concatenated results and loss
y2: torch.Tensor = model(x)
print(torch.cat((y1, y2), dim=1))  
loss(y1, y2)


tensor([[0.7652, 0.7652],
        [0.6517, 0.6517],
        [0.2455, 0.2455],
        [0.6973, 0.6973]], grad_fn=<CatBackward0>)
loss = 0.0000000000%


In [223]:
# Get the output after evolution and pruning
with torch.no_grad():
    for layer in model.layers:
        if isinstance(layer, Linear) or str(type(layer)) == "<class 'layers.Linear'>":
            layer.weight.data *= 0.03
            if layer.bias is not None:
                layer.bias.data *= 0.03
    # for layer in model.activation:
    #     if isinstance(layer, ActiSwitch) or str(type(layer)) == "<class 'layers.ActiSwitch'>":
    #         layer.weight.data.copy_(torch.tensor(1.0))

model.prune(0.01)
# summary(model, input_size=(5,))
model.summary()

Model Summary:
-------------------------------------------------------------------------------------
Layer      Output Shape                  Parameters     Activation     
-------------------------------------------------------------------------------------
Layer 1    (batch_size, 39)              234            ActiSwitch(ReLU, 0.00%)
Layer 2    (batch_size, 12)              480            ActiSwitch(ReLU, 0.00%)
Layer 3    (batch_size, 6)               78             ActiSwitch(ReLU, 0.00%)
Layer 4    (batch_size, 10)              70             ActiSwitch(ReLU, 0.00%)
Layer 5    (batch_size, 5)               55             ActiSwitch(ReLU, 0.00%)
Layer 6    (batch_size, 7)               42             ActiSwitch(ReLU, 0.00%)
Layer 7    (batch_size, 9)               72             ActiSwitch(ReLU, 0.00%)
Layer 8    (batch_size, 3)               30             ActiSwitch(ReLU, 0.00%)
Layer 9    (batch_size, 7)               28             ActiSwitch(ReLU, 0.00%)
Layer 10   (batch_siz

### How Crossover works

In [224]:
# Create parent networks with specified architectures
parent1 = ATNetwork([10, 2, 5, 1])
parent2 = ATNetwork([10, 2, 4, 3, 1])

# Initialize the GeneticAlgorithm instance
ga = ATGEN(population_size=10, layers=[8, 1, 4])

# Perform crossover
child = ga.crossover(parent1, parent2)

# Print the child network architecture and parameters
# summary(child, (10,))
model.summary()

# ga.run_generation(dummy_fitness_function, 10)

Model Summary:
-------------------------------------------------------------------------------------
Layer      Output Shape                  Parameters     Activation     
-------------------------------------------------------------------------------------
Layer 1    (batch_size, 39)              234            ActiSwitch(ReLU, 0.00%)
Layer 2    (batch_size, 12)              480            ActiSwitch(ReLU, 0.00%)
Layer 3    (batch_size, 6)               78             ActiSwitch(ReLU, 0.00%)
Layer 4    (batch_size, 10)              70             ActiSwitch(ReLU, 0.00%)
Layer 5    (batch_size, 5)               55             ActiSwitch(ReLU, 0.00%)
Layer 6    (batch_size, 7)               42             ActiSwitch(ReLU, 0.00%)
Layer 7    (batch_size, 9)               72             ActiSwitch(ReLU, 0.00%)
Layer 8    (batch_size, 3)               30             ActiSwitch(ReLU, 0.00%)
Layer 9    (batch_size, 7)               28             ActiSwitch(ReLU, 0.00%)
Layer 10   (batch_siz

In [225]:
model = ATNetwork([1, 4, 1])
optim = AdamW(model.parameters(), lr=0.01)
# summary(model, (1,))
model.summary()

x = torch.randn(10, 1)
y_real = torch.randn(10, 1)
epochs = 1_000

Model Summary:
-------------------------------------------------------------------------------------
Layer      Output Shape                  Parameters     Activation     
-------------------------------------------------------------------------------------
Layer 1    (batch_size, 4)               8              ActiSwitch(ReLU, 0.00%)
Layer 2    (batch_size, 1)               5              Pass           
-------------------------------------------------------------------------------------
Total Parameters:        13             


In [226]:
for i in range(epochs):
    y_pred = model(x)
    loss = F.mse_loss(y_pred, y_real)
    optim.zero_grad()
    loss.backward()
    optim.step()
print("Loss:", loss.item())


Loss: 1.378637671470642


In [227]:
for i in range(epochs):
    y_pred = model(x)
    loss = F.mse_loss(y_pred, y_real)
    optim.zero_grad()
    loss.backward()
    optim.step()
print("Loss:", loss.item())

Loss: 1.3776894807815552


In [228]:
for _ in range(5):
    model.evolve_network()
for _ in range(100):
    model.evolve_layer()
optim = AdamW(model.parameters(), lr=0.01)
# summary(model, (1,))
model.summary()


Model Summary:
-------------------------------------------------------------------------------------
Layer      Output Shape                  Parameters     Activation     
-------------------------------------------------------------------------------------
Layer 1    (batch_size, 12)              24             ActiSwitch(ReLU, 0.00%)
Layer 2    (batch_size, 23)              299            ActiSwitch(ReLU, 0.00%)
Layer 3    (batch_size, 20)              480            ActiSwitch(ReLU, 0.00%)
Layer 4    (batch_size, 21)              441            ActiSwitch(ReLU, -1659.06%)
Layer 5    (batch_size, 20)              440            ActiSwitch(ReLU, 0.00%)
Layer 6    (batch_size, 19)              399            ActiSwitch(ReLU, 0.00%)
Layer 7    (batch_size, 1)               20             Pass           
-------------------------------------------------------------------------------------
Total Parameters:        2103           


In [229]:
print(model.layers[5].weight)

Parameter containing:
tensor([[ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          0.0

In [230]:
for i in range(epochs):
    y_pred = model(x)
    loss = F.mse_loss(y_pred, y_real)
    optim.zero_grad()
    loss.backward()
    optim.step()
print("Loss:", loss.item())

Loss: 0.1641933023929596


In [231]:
print(model.layers[5].weight)

Parameter containing:
tensor([[ 9.8961e-01,  1.1684e-01,  7.1409e-02,  1.1846e-01,  1.9196e-02,
          8.0293e-02, -8.0308e-02, -6.4828e-02,  6.1465e-02, -1.4453e-01,
         -4.8063e-02,  1.5090e-01,  7.1782e-03,  9.3539e-02,  5.5773e-02,
         -5.1169e-02, -2.7981e-02,  5.2644e-03,  1.5961e-01, -1.3182e-01],
        [ 8.9619e-02,  1.0210e+00,  7.8777e-02,  1.0878e-01,  1.2618e-02,
          8.4600e-02, -8.0633e-02, -6.5061e-02,  6.1697e-02, -1.5741e-01,
         -4.9691e-02,  1.5787e-01,  2.0314e-03,  1.0104e-01,  5.8417e-02,
         -6.1539e-02, -3.1300e-02,  1.2225e-02,  1.6842e-01, -1.3105e-01],
        [ 9.1263e-02,  1.2246e-01,  9.7824e-01,  1.2456e-01,  1.9880e-02,
          9.5953e-02, -7.4527e-02, -7.6565e-02,  6.0686e-02, -1.7221e-01,
         -5.0984e-02,  1.6488e-01,  1.1754e-03,  8.8362e-02,  6.4483e-02,
         -6.3099e-02, -3.4460e-02,  1.7441e-02,  1.6932e-01, -1.3428e-01],
        [ 7.6079e-02,  1.0784e-01,  7.3396e-02,  1.0180e+00,  1.4933e-02,
          1.0